In [1]:
import torch
import torch.nn as nn

class TestModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

model = TestModel()
traced_model = torch.jit.trace(model, torch.randn(1, 128))

# Save the TorchScript model
torch.jit.save(traced_model, "model.pt")


In [2]:
!torch-mlir-opt --torchscript model.pt -o model.mlir

Traceback (most recent call last):
  File "/opt/anaconda3/envs/mlir_env/bin/torch-mlir-opt", line 33, in <module>
    sys.exit(load_entry_point('torch-mlir==0.0.1', 'console_scripts', 'torch-mlir-opt')())
  File "/opt/anaconda3/envs/mlir_env/bin/torch-mlir-opt", line 25, in importlib_load_entry_point
    return next(matches).load()
  File "/opt/anaconda3/envs/mlir_env/lib/python3.10/importlib/metadata/__init__.py", line 171, in load
    module = import_module(match.group('module'))
  File "/opt/anaconda3/envs/mlir_env/lib/python3.10/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1050, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 992, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "<frozen importlib._bootstrap>", line 1050, in _

In [ ]:
!torch-mlir-opt --convert-torch-to-tensor model.mlir -o tensor.mlir

In [ ]:
!mlir-opt --convert-linalg-to-affine-loops tensor.mlir -o affine.mlir

In [ ]:
!iree-compile affine.mlir -backend=llvm-cpu -o mobile_model.vmfb